# Converting the MNE "somato-data" to BIDS format

## Requirements

The python dependencies to run this notebook are:
- jupyter
- numpy
- scipy
- mne<0.19
- mne-bids
- nibabel


## Helpful links

- https://github.com/mne-tools/mne-bids

## Copyright
```Text
# Authors: Stefan Appelhoff <stefan.appelhoff@mailbox.org>
#
# License: BSD (3-clause)
```


In [ ]:
import os
import os.path as op
import shutil as sh
import json
import warnings

import numpy as np
import mne
import mne.datasets.somato as somato
from mne_bids import write_raw_bids, make_bids_basename, write_anat
from mne_bids.utils import print_dir_tree

# Somato data prior to conversion

In [ ]:
# Print the directory tree
print_dir_tree(somato.data_path())

# Convert to BIDS

In [ ]:
# Path to the raw data of somato dataset
fif_path = op.join(somato.data_path(), 'MEG', 'somato')

In [ ]:
# Load the raw data file
raw = mne.io.read_raw_fif(op.join(fif_path, 'sef_raw_sss.fif'))

In [ ]:
# Set the sex of the participant to male
# Subject sex (0=unknown, 1=male, 2=female).
raw.info['subject_info']['sex'] = raw.info['subject_info'].get('sex', 1)

In [ ]:
# Get the events, and assign an ID
events = mne.find_events(raw)
event_id = dict()
for val in np.unique(events[:, -1]):
    event_id['somato_event{}'.format(val)] = val
    
event_id

In [ ]:
# Make a bids name for the raw file
bids_basename = make_bids_basename(subject='01', task='somato')

In [ ]:
# Make a location for the BIDS formatted data
somato_parent, somato_name = op.split(somato.data_path())
output_path = op.join(somato_parent, somato_name + '-bids')

In [ ]:
# Write the data
write_raw_bids(raw, bids_basename, output_path, events, event_id, overwrite=True)

In [ ]:
# Edit dataset_description.json to include link to dataset
dataset_description_json = op.join(output_path, 'dataset_description.json')
with open(dataset_description_json, 'r') as fin:
    jsondict = json.load(fin)
    
# Edit some fields we know of
jsondict['Name'] = 'MNE-somato-data-bids'
jsondict['Authors'] = ['Lauri Parkkonen']
jsondict['License'] = 'PDDL, see: https://opendatacommons.org/licenses/pddl/'
jsondict['ReferencesAndLinks'] = ['http://martinos.org/mne/stable/manual/datasets_index.html?#somatosensory']
jsondict['Acknowledgements'] = "Stefan Appelhoff, Alexandre Gramfort, and Mainak Jas formatted these data to BIDS."
    
    
# overwrite the json, updating it
with open(dataset_description_json, 'w') as fout:
    json.dump(jsondict, fout,indent=2, sort_keys=True)

In [ ]:
# Write anatomical data to BIDS
# we take the original T1 from the freesurfer directory
t1w = op.join(somato.data_path(), 'subjects', 'somato', 'mri', 'T1.mgz')

# we also take the trans file, and use it to write the coordinates of 
# anatomical landmarks to a T1w.json file
trans = op.join(somato.data_path(), 'MEG', 'somato', 'sef_raw_sss-trans.fif')

# Copy over the MRI, convert it to NIfTI format, and write the anatomical landmarks
# in voxel coordinates
write_anat(output_path, subject='01', t1w=t1w, trans=trans, raw=raw,
           overwrite=True, verbose=True)

In [ ]:
# Show what the converted data looks like so far
print_dir_tree(output_path)

# Add derivatives

some files cannot be specified in BIDS yet. We add these to derivatives


In [ ]:
# General derivatives directory
derivatives_dir =op.join(output_path, 'derivatives')
if not op.exists(derivatives_dir):
    os.makedirs(derivatives_dir)

##### Notes on freesurfer

we *cannot* simply copy over the freesurfer files from the non-BIDS somato data, because we need to rename the subject to '01' instead of 'somato'

###### Example workflow (bash)

First run freesurfer:

```bash
$ export SUBJECTS_DIR=/home/username/subjects

$ echo $SUBJECTS_DIR
/home/username/subjects

$ recon-all -all -i /home/username/bids_MNE-somato-data/sub-01/anat/sub-01_T1w.nii.gz -s 01
```

Then your `$SUBJECTS_DIR` should contain a folder `01` with the somato MRI data derivatives and you can successfully run the following cell.

In [ ]:
# Make a freesurfer directory. 
subjects_dir = op.join(derivatives_dir, 'freesurfer', 'subjects')
if not op.exists(subjects_dir):
    os.makedirs(subjects_dir)
    
# For more info, see the markdown cell above "notes on freesurfer"    
local_subjects_dir = os.getenv('SUBJECTS_DIR', None)
if local_subjects_dir is not None:
    freesurfer_data_to_copy_overop.join(local_subjects_dir, '01')
    sh.copytree(freesurfer_data_to_copy_over, subjects_dir)
else:
    warnings.warn('\n\n**NOT** copying the freesurfer data. '
                  'Please read the instructions in the cell above.')
    
    

In [ ]:
# Make a directory for our subject and move the forward model there
forward_dir = op.join(derivatives_dir, 'sub-01')
if not op.exists(forward_dir):
    os.makedirs(forward_dir)
    
# copy it, overwriting old if it's there
old_forward = op.join(somato.data_path(), 'MEG', 'somato', 'somato-meg-oct-6-fwd.fif')
bids_forward = op.join(forward_dir, 'sub-01_task-somato-fwd.fif')
sh.copyfile(old_forward, bids_forward)

In [ ]:
# Show what the converted data looks like so far
print_dir_tree(output_path)

# Create READMEs and copy over this code as well

In [ ]:
# Prepare the text for the central README
txt1 = """MNE-somato-data-bids
====================

This dataset contains the MNE-somato-data in BIDS format.

The conversion can be reproduced through the jupyter notebook stored in the
/code directory of this dataset. See the README in that directory.

The /derivaties directory contains the outputs of running the freesurfer pipeline
`recon-all` on the MRI data with no additional commandline options (only defaults were used).

It also contains, the forward model *-fwd.fif, which was produced using the source space
definition, a -trans.fif file, and the boundary element model / conductor model that live
in freesurfer/subjects/01/bem/*-bem-sol.fif.

The -trans.fif file is not saved, but can be recovered from the anatomical landmarks
in the sub-01/anat/T1w.json file and MNE-BIDS' function `get_head_mri_transform`.

See: https://github.com/mne-tools/mne-bids for more information.


Note on freesurfer
==================
the freesurfer pipeline `recon-all` was run new for the sake of converting the somato
data to BIDS format. This needed to be done to change the "somato" subject name to 
the BIDS subject label "01".


"""

In [ ]:
readme_fname = op.join(output_path, 'README')

with open(readme_fname, 'w') as fout:
    print(txt1, file=fout)

In [ ]:
# Prepare a /code directory and write a README
code_dir = op.join(output_path, 'code')
if not op.exists(code_dir):
    os.makedirs(code_dir)

txt2 = """`convert_somato_data.ipynb` is a [jupyter notebook](https://jupyter.org/)
that helps to convert the MNE-somato-data into [BIDS](bids.neuroimaging.io) format.

To run this notebook, you can follow the instructions here: https://jupyter.org/install
"""

readme_fname = op.join(code_dir, 'README')

with open(readme_fname, 'w') as fout:
    print(txt2, file=fout)


In [ ]:
# Finally, copy over this jupyter notebook as well
fname = 'convert_somato_data.ipynb'
fpath = os.path.dirname(os.path.realpath('__file__'))
fpath = op.join(fpath, fname)

sh.copyfile(fpath, op.join(code_dir, fname))

In [ ]:
# Show what the converted data looks like so far
print_dir_tree(output_path)